In [31]:
import pandas as pd
import numpy as np

In [32]:
df_name_list = ["BET", "BUX", "EGX30", "NGXINDX", "Warsaw Stock Exchange WIG Index"]

In [33]:
for name in df_name_list:
    df_daily = pd.read_excel(name + " daily prices.xlsx")
    df_daily["Date"] = df_daily["Date"].astype("datetime64[D]")

    last_trade_date = []
    O_Date = df_daily.loc[0, "Date"]
    Price = df_daily.loc[0, "Last Price"]
    last_trade_date.append((O_Date.date(), Price))
    for i in range(len(df_daily)):
        Date = df_daily.loc[i, "Date"]
        Price = df_daily.loc[i, "Last Price"]
        if Date.month != O_Date.month or Date.year != O_Date.year:
            O_Date = Date
            last_trade_date.append((O_Date.date(), Price))
    
    temp1, temp2 = zip(*last_trade_date)
    df_monthly = pd.DataFrame({"Date": temp1, "Last_Price":temp2})

    df_mom_change = df_monthly.copy()
    for i in range(len(df_mom_change)-1):
        df_mom_change.loc[i, "mom%change"] = (df_mom_change.loc[i, "Last_Price"] - df_mom_change.loc[i+1, "Last_Price"]) / df_mom_change.loc[i+1, "Last_Price"] * 100
        
    df_mo24mma_change = df_monthly.copy()
    df_mo24mma_change = df_mo24mma_change.sort_values("Date")
    df_mo24mma_change["24mma"] = df_mo24mma_change["Last_Price"].rolling(24).mean()
    df_mo24mma_change = df_mo24mma_change.sort_values("Date", ascending=False)
    for i in range(len(df_mo24mma_change)-24):
        df_mo24mma_change.loc[i, "mo24mma%change"] = df_mo24mma_change.loc[i, "Last_Price"]/ df_mo24mma_change.loc[i+1, "24mma"] 

    df_Volatility = df_daily[["Date", "Last Price"]]
    df_Volatility = df_Volatility.sort_values("Date")
    df_Volatility["20 days average"] = df_Volatility["Last Price"].rolling(20).mean()
    df_Volatility["Square of Difference"] = np.square(df_Volatility["Last Price"] - df_Volatility["20 days average"])
    df_Volatility["Square of Difference 20 days average"] = df_Volatility["Square of Difference"].rolling(20).mean()
    df_Volatility["Volatility"] = np.sqrt(df_Volatility["Square of Difference 20 days average"])
    df_Volatility = df_Volatility.sort_values("Date", ascending = False)

    df_monthly_Volatility = df_Volatility.groupby(pd.PeriodIndex(df_Volatility['Date'], freq="M"))['Volatility'].mean()

    
    df_mom_change.to_csv(name + "_mom%change.csv")
    df_mo24mma_change.to_csv(name + "_mo24mma_change.csv")
    df_Volatility.to_csv(name + "_Volatility.csv")
    df_monthly_Volatility.to_csv(name + "monthly_Volatility.csv")


In [34]:
df1 = pd.read_csv("Warsaw Stock Exchange WIG Index_Volatility.csv")
df1.groupby(pd.PeriodIndex(df1['Date'], freq="M"))['Volatility'].mean()


Date
2000-12            NaN
2001-01            NaN
2001-02     460.118264
2001-03     896.558353
2001-04     481.096630
              ...     
2023-06    1464.486301
2023-07    1659.624418
2023-08    2007.966817
2023-09    1499.400729
2023-10    1474.624715
Freq: M, Name: Volatility, Length: 275, dtype: float64